# Optimizing the design and operation of a two-nodal energy system with networks
In this study, we want to look at a two-nodal system in a brownfield manner. We assume two fictous regions with a heat demand and an electricity demand and we want to study the decarbonization of these two regions and the impact of a carbon tax. First, we will optimize the design and operation of the system without a carbon tax and then study the effect of a carbon tax. The topology of the energy system is depicted below. Brown rectangles mean that the technology is already installed, and green rectangles mean that the technology can be newly built. Both colours mean that the technology can be expanded.

We assume that there are two nodes: one depicting a city with no options to install wind turbines and no power plant and a rural node with a gas-fired power plant and the option to install wind turbines.

<div>
<img src="figures/network_topology.png" width="500"/>
</div>

## Create templates
We set the input data path and in this directory we can add input data templates for the model configuration and the topology with the function create_optimization_templates.


In [ ]:
import adopt_net0 as adopt
import json
from pathlib import Path
import os
import pandas as pd
import numpy as np

# Create folder for results
results_data_path = Path("./userData")
results_data_path.mkdir(parents=True, exist_ok=True)
# Create input data path and optimization templates
input_data_path = Path("./caseStudies/network")
input_data_path.mkdir(parents=True, exist_ok=True)
adopt.create_optimization_templates(input_data_path)

## Adapt Topology
We need to adapt the topology as well as the model configuration file to our case study. This can be done either in the file itself (Topology.json) or, as we do it here via some lines of code.
For the topology, we need to change the following:
- Change nodes: city and urban area
- Change carriers: electricity, heat and natural gas (we need to add hydrogen as a carrier as well, as the gas turbine also allows for a hydrogen input, even though we do not use it in this case study)
- Change investment periods: period1
- The options regarding the time frame we can leave at the default (one year with hourly operation)

In [ ]:
# Load json template
with open(input_data_path / "Topology.json", "r") as json_file:
    topology = json.load(json_file)
# Nodes
topology["nodes"] = ["city", "rural"]
# Carriers:
topology["carriers"] = ["electricity", "heat", "gas", "hydrogen"]
# Investment periods:
topology["investment_periods"] = ["period1"]
# Save json template
with open(input_data_path / "Topology.json", "w") as json_file:
    json.dump(topology, json_file, indent=4)

## Adapt Model Configurations
The model configuration we leave as it is, except spcifying a number of typical days to speed up the optimization (this system also solves on full resolution, but here, we want to only show how the model works at the expense of precision). Additionally, we set the MILP gap to 2%.
- Change the number of typical days to 30 and select time aggregation method 1 (see [here](https://adopt-net0.readthedocs.io/en/latest/advanced_topics/time_aggregation.html#clustering-into-typical-days))
- Change the MILP gap to 2%

In [ ]:
# Load json template
with open(input_data_path / "ConfigModel.json", "r") as json_file:
    configuration = json.load(json_file)
# Set time aggregation settings:
configuration["optimization"]["typicaldays"]["N"]["value"] = 30
configuration["optimization"]["typicaldays"]["method"]["value"] = 1
# Set MILP gap
configuration["solveroptions"]["mipgap"]["value"] = 0.02
# Save json template
with open(input_data_path / "ConfigModel.json", "w") as json_file:
    json.dump(configuration, json_file, indent=4)

## Define new and existing technologies at each node
First, we create the required folder structure based on the Topology.json file, add locations. We can then show all available technologies.

In [ ]:
adopt.create_input_data_folder_template(input_data_path)

# Define node locations (here two exemplary location in the Netherlands)
node_location = pd.read_csv(input_data_path / "NodeLocations.csv", sep=';', index_col=0, header=0)
node_lon = {'city': 5.1214, 'rural': 5.24}
node_lat = {'city': 52.0907, 'rural': 51.9561}
node_alt = {'city': 5, 'rural': 10}
for node in ['city', 'rural']:
    node_location.at[node, 'lon'] = node_lon[node]
    node_location.at[node, 'lat'] = node_lat[node]
    node_location.at[node, 'alt'] = node_alt[node]

node_location = node_location.reset_index()
node_location.to_csv(input_data_path / "NodeLocations.csv", sep=';', index=False)

adopt.show_available_technologies()

... And we can add the technologies that we need to the Technologies.json file at the respective nodes and copy over the required technology data

In [ ]:
# Add required technologies for node 'city'
with open(input_data_path / "period1" / "node_data" / "city" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["HeatPump_AirSourced", "Storage_Battery", "Photovoltaic"]
technologies["existing"] = {"Boiler_Small_NG": 1000}

with open(input_data_path / "period1" / "node_data" / "city" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)

# Add required technologies for node 'rural'
with open(input_data_path / "period1" / "node_data" / "rural" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["HeatPump_AirSourced", "Storage_Battery", "Photovoltaic", "WindTurbine_Onshore_4000"]
technologies["existing"] = {"Boiler_Small_NG": 350, "GasTurbine_simple": 1000}

with open(input_data_path / "period1" / "node_data" / "rural" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)

# Copy over technology files
adopt.copy_technology_data(input_data_path)

## Change maximum sizes of heat pumps and boilers
As these technologies are by default set to a household level, we need to change the maximum sizes to make them suitable to meet a larger energy demand

In [ ]:
for node in ["city", "rural"]:
    with open(input_data_path / "period1" / "node_data" / node / "technology_data"/ "Boiler_Small_NG.json", "r") as json_file:
        tec_data = json.load(json_file)
        tec_data["size_max"] = 2000

    with open(input_data_path / "period1"  /  "node_data" /node / "technology_data"/ "Boiler_Small_NG.json", "w") as json_file:
        json.dump(tec_data, json_file, indent=4)
    
    with open(input_data_path / "period1" /  "node_data" / node / "technology_data"/ "HeatPump_AirSourced.json", "r") as json_file:
        tec_data = json.load(json_file)
        tec_data["size_max"] = 3000
    with open(input_data_path / "period1" /  "node_data" / node / "technology_data"/ "HeatPump_AirSourced.json", "w") as json_file:
        json.dump(tec_data, json_file, indent=4)
 

## Define the existing and new electricity network between the two nodes
We can see available networks that ship with the model with:

In [ ]:
adopt.show_available_networks()

In this case, we will use 'electricityOnshore' for both the existing and new network:

In [ ]:
# Add networks
with open(input_data_path / "period1" / "Networks.json", "r") as json_file:
    networks = json.load(json_file)
networks["new"] = ["electricityOnshore"]
networks["existing"] = ["electricityOnshore"]

with open(input_data_path / "period1" / "Networks.json", "w") as json_file:
    json.dump(networks, json_file, indent=4)

Now we need to specify the network topologies for both the existing and the new network and copy them in the respective directory. This is easier done manually, but here we do it using Python.
New networks need the following files (size_max_arcs is optional):
- connection.csv (1 or 0)
- distance.csv (distance in km): 50km between the two nodes
  
Existing networks need the following files:
- connection.csv (1 or 0)
- distance.csv (distance in km): 50km between the two nodes
- size.csv (size in MW): 1GW connection

In [ ]:
# Make a new folder for the existing network
os.makedirs(input_data_path / "period1" / "network_topology" / "existing" / "electricityOnshore", exist_ok=True)

print("Existing network")
# Use the templates, fill and save them to the respective directory
# Connection
connection = pd.read_csv(input_data_path / "period1" / "network_topology" / "existing" / "connection.csv", sep=";", index_col=0)
connection.loc["city", "rural"] = 1
connection.loc["rural", "city"] = 1
connection.to_csv(input_data_path / "period1" / "network_topology" / "existing" / "electricityOnshore" / "connection.csv", sep=";")
print("Connection:", connection)

# Delete the template
os.remove(input_data_path / "period1" / "network_topology" / "existing" / "connection.csv")

# Distance
distance = pd.read_csv(input_data_path / "period1" / "network_topology" / "existing" / "distance.csv", sep=";", index_col=0)
distance.loc["city", "rural"] = 50
distance.loc["rural", "city"] = 50
distance.to_csv(input_data_path / "period1" / "network_topology" / "existing" / "electricityOnshore" / "distance.csv", sep=";")
print("Distance:", distance)

# Delete the template
os.remove(input_data_path / "period1" / "network_topology" / "existing" / "distance.csv")

# Size
size = pd.read_csv(input_data_path / "period1" / "network_topology" / "existing" / "size.csv", sep=";", index_col=0)
size.loc["city", "rural"] = 1000
size.loc["rural", "city"] = 1000
size.to_csv(input_data_path / "period1" / "network_topology" / "existing" / "electricityOnshore" / "size.csv", sep=";")
print("Size:", size)

# Delete the template
os.remove(input_data_path / "period1" / "network_topology" / "existing" / "size.csv")


print("New network")
# Make a new folder for the new network
os.makedirs(input_data_path / "period1" / "network_topology" / "new" / "electricityOnshore", exist_ok=True)


# max size arc
arc_size = pd.read_csv(input_data_path / "period1" / "network_topology" / "new" / "size_max_arcs.csv", sep=";", index_col=0)
arc_size.loc["city", "rural"] = 3000
arc_size.loc["rural", "city"] = 3000
arc_size.to_csv(input_data_path / "period1" / "network_topology" / "new" / "electricityOnshore" / "size_max_arcs.csv", sep=";")
print("Max size per arc:", arc_size)

# Use the templates, fill and save them to the respective directory
# Connection
connection = pd.read_csv(input_data_path / "period1" / "network_topology" / "new" / "connection.csv", sep=";", index_col=0)
connection.loc["city", "rural"] = 1
connection.loc["rural", "city"] = 1
connection.to_csv(input_data_path / "period1" / "network_topology" / "new" / "electricityOnshore" / "connection.csv", sep=";")
print("Connection:", connection)

# Delete the template
os.remove(input_data_path / "period1" / "network_topology" / "new" / "connection.csv")

# Distance
distance = pd.read_csv(input_data_path / "period1" / "network_topology" / "new" / "distance.csv", sep=";", index_col=0)
distance.loc["city", "rural"] = 50
distance.loc["rural", "city"] = 50
distance.to_csv(input_data_path / "period1" / "network_topology" / "new" / "electricityOnshore" / "distance.csv", sep=";")
print("Distance:", distance)

# Delete the template
os.remove(input_data_path / "period1" / "network_topology" / "new" / "distance.csv")

# Delete the max_size_arc template
os.remove(input_data_path / "period1" / "network_topology" / "new" / "size_max_arcs.csv")

## Copy over network data
Copy over network data, change cost data.

In [ ]:
adopt.copy_network_data(input_data_path)

with open(input_data_path / "period1" / "network_data"/ "electricityOnshore.json", "r") as json_file:
    network_data = json.load(json_file)

network_data["Economics"]["gamma2"] = 40000
network_data["Economics"]["gamma4"] = 300

with open(input_data_path / "period1" / "network_data"/ "electricityOnshore.json", "w") as json_file:
    json.dump(network_data, json_file, indent=4)

## Define demand, climate data, import limits, import prices

In [ ]:
# Read hourly data from Excel (Demand profiles are provided with the package)
rural_hourly_data = adopt.load_network_rural_data()
city_hourly_data = adopt.load_network_city_data()

# Save the hourly data to the carrier's file in the case study folder
# electricity demand and price
hourly_data = {
    'city': city_hourly_data,
    'rural': rural_hourly_data
}

el_demand = {}
heat_demand = {}

for node in ['city', 'rural']:
    el_demand[node] = hourly_data[node].iloc[:, 1]
    heat_demand[node] = hourly_data[node].iloc[:, 0]
    adopt.fill_carrier_data(input_data_path, value_or_data=el_demand[node], columns=['Demand'], carriers=['electricity'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=heat_demand[node], columns=['Demand'], carriers=['heat'], nodes=[node])

    # Set import limits/cost
    adopt.fill_carrier_data(input_data_path, value_or_data=4000, columns=['Import limit'], carriers=['gas'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=2000, columns=['Import limit'], carriers=['electricity'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=0.25, columns=['Import emission factor'], carriers=['electricity'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=40, columns=['Import price'], carriers=['gas'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=120, columns=['Import price'], carriers=['electricity'], nodes=[node])

# Define climate data
adopt.load_climate_data_from_api(input_data_path)

## Run without carbon costs

In [ ]:
m = adopt.ModelHub()
m.read_data(input_data_path)
m.quick_solve()

## First results: technology sizes
The results of the optimizations, which are saved in the userData folder, can be (partially) visualized using the provided visualization platform (https://resultvisualization.streamlit.app/). Here we just report some screenshots, specifically the sizes (all in MW) of the technologies installed in the network. It can be noticed that, in addition to the existing technologies (gas boiler in the city node, gas boiler and gas plant in the rural node), it is cost-efficient to install heat pumps in both nodes and more than 5 GW of PV in the city node. Note that the heat pump sizes seem quite small, but it is given in terms of electricity input, so the heat output needs to be multiplied by the COP.

### Sizes heating technologies
<div>
<img src="figures/network_heating_tecs_nocarbontax.png" width="700"/>
</div>

### Sizes electricity technologies
<div>
<img src="figures/network_electricity_tecs_nocarbontax.png" width="700"/>
</div>

## First results: network
From a network perspective, no transmission capacity has been added. The electricity exchange happens only from the rural to the city node (~1.1 TWh exchanged over 1 year).

<div>
<img src="figures/network_flow_nocarbontax.png" width="300"/>
</div>

## Run with carbon costs

In [ ]:
# Set carbon emission price
carbon_price = np.ones(8760)*100
for node in ["city", "rural"]:
    carbon_cost_path = "./caseStudies/network/period1/node_data/" + node + "/CarbonCost.csv"
    carbon_cost_template = pd.read_csv(carbon_cost_path, sep=';', index_col=0, header=0)
    carbon_cost_template['price'] = carbon_price
    carbon_cost_template = carbon_cost_template.reset_index()
    carbon_cost_template.to_csv(carbon_cost_path, sep=';', index=False)

m = adopt.ModelHub()
m.read_data(input_data_path)
m.quick_solve()

## Results after the introduction of a CO2 tax: technology sizes
The introduction of a CO2 tax leads to higher deployment of PV in the city node(~8.3GW vs ~5.3GW), with consequent reduced use of the gas turbine in the rural node. Regarding the heating technologies, the size of the heat pump increases by a few MW in both the nodes

### Sizes heating technologies
<div>
<img src="figures/network_heating_tecs_carbontax.png" width="700"/>
</div>

### Sizes electricity technologies
<div>
<img src="figures/network_electricity_tecs_carbontax.png" width="700"/>
</div>


## Network operations

When the CO2 tax is included, the electricity flow changes direction and the electricity is exported from the PV "rich" city to the rural area, where the use of the gas turbine has become more expensive. Around 1.4 TWh of electricity are exported.

<div>
<img src="figures/network_flow_carbontax.png" width="300"/>
</div>